In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
from itertools import product
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
cats = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
shops = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')

In [ ]:
plt.figure(figsize=(10,4))
plt.xlim(-100, 3000)
sns.boxplot(x=train.item_cnt_day)

plt.figure(figsize=(10,4))
plt.xlim(train.item_price.min(), train.item_price.max()*1.1)
sns.boxplot(x=train.item_price)

In [ ]:
train = train[train.item_cnt_day< 2000]
train = train[train.item_price < 100000]

In [ ]:
train.loc[train.item_price < 0, 'item_price'] = train['item_price'].mean()

In [ ]:
# Якутск Орджоникидзе, 56
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

In [ ]:
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
shops = shops[['shop_id','city_code']]

cats['split'] = cats['item_category_name'].str.split('-')
cats['type'] = cats['split'].map(lambda x: x[0].strip())
cats['type_code'] = LabelEncoder().fit_transform(cats['type'])
# if subtype is nan then type
cats['subtype'] = cats['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
cats['subtype_code'] = LabelEncoder().fit_transform(cats['subtype'])
cats = cats[['item_category_id','type_code', 'subtype_code']]

items.drop(['item_name'], axis=1, inplace=True)

In [ ]:
test_shops = test.shop_id.unique()
test_items = test.item_id.unique()
train = train[train.shop_id.isin(test_shops)]
train = train[train.item_id.isin(test_items)]

In [ ]:
# create all combinations
block_shop_combi = pd.DataFrame(list(product(np.arange(34), test_shops)), columns=['date_block_num','shop_id'])
shop_item_combi = pd.DataFrame(list(product(test_shops, test_items)), columns=['shop_id','item_id'])
all_combi = pd.merge(block_shop_combi, shop_item_combi, on=['shop_id'], how='inner')


In [ ]:
train_base = pd.merge(all_combi, train, on=['date_block_num','shop_id','item_id'], how='left')
train_base['item_cnt_day'].fillna(0, inplace=True)
train_grp = train_base.groupby(['date_block_num','shop_id','item_id']).sum()

In [ ]:
train_grp['item_cnt_day'].clip(0, 20, inplace=True)
train_grp.reset_index(inplace = True)

In [ ]:
train_grp.drop('item_price', axis = 1, inplace = True)

In [ ]:
train_grp

In [ ]:
test

In [ ]:
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num']

In [ ]:
test.set_index('ID', inplace = True)

In [ ]:
test

In [ ]:
cols = ['date_block_num','shop_id','item_id']
train_grp = pd.concat([train_grp, test], ignore_index=True, sort=False, keys=cols)
train_grp.fillna(0, inplace=True) # 34 month

In [ ]:
train_grp

In [ ]:
train_grp = pd.merge(train_grp, shops, on=['shop_id'], how='left')
train_grp = pd.merge(train_grp, items, on=['item_id'], how='left')
train_grp = pd.merge(train_grp, cats, on=['item_category_id'], how='left')

In [ ]:
train_grp

In [ ]:
missing = train_grp.isnull().sum()
missing

In [ ]:
X_train = train_grp[train_grp.date_block_num <= 33].drop(['item_cnt_day'], axis=1)
y_train = train_grp[train_grp.date_block_num <= 33]['item_cnt_day']
X_test = train_grp[train_grp.date_block_num == 34].drop(['item_cnt_day'], axis=1)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression().fit(X_train, y_train)
y_predict = model.predict(X_test).clip(0,20)


In [ ]:
submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_day": y_predict
})
submission.to_csv('submit.csv', index=False)